https://www2.seas.gwu.edu/~simhaweb/alg/modules/module8/module8.html

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.sparse.csgraph import shortest_path
from scipy.sparse import csr_matrix
import sys
import time

In [2]:
class vertex:
    index = 0
    weight = 0
    def __init__(self,index,weight):
        self.index = index
        self.weight = weight

#Generate positive directed-graph
#Generates {adj_matrix} and {adj_list} of size (num_nodes,num_nodes)
def generate_graph(num_nodes):
    # Generate Adjacent Matrix
    adj_matrix = np.random.randint(0,num_nodes,(num_nodes,num_nodes))
    np.fill_diagonal(adj_matrix,0)
    
    # Convert Adjacent Matrix to List
    adj_list = []
    
    col_index = 0
    for row in adj_matrix:
        node_ll = []
        vertex_index = 0
        
        for node in row:
            if node != 0:
                node_ll.append(vertex(vertex_index,node))
            vertex_index+=1
            
        adj_list.append(node_ll)
        col_index+=1
    
    return adj_matrix, adj_list

# Generate positive directed-graph with increasing vertices of random values
# returns {num_elements}*{num_elements} graphs
def generate_graphs_fixed_element(num_elements):
    adj_matrices = []
    adj_lists = []

    # Generate Zero Matrix
    adj_matrix = np.zeros((num_elements,num_elements))
    row_index = 0
    for i in adj_matrix:
        for node in range(len(i)):
            if row_index != node:
                #Change current point vertice value to random number
                adj_matrix[row_index][node] = np.random.randint(1,num_elements)
                adj_matrices.append(adj_matrix.copy())

                #Convert adj matrix to adj list
                adj_list = []
                col_index = 0
                for row in adj_matrix:
                    node_ll = []
                    vertex_index = 0
                    for node in row:
                        if node != 0:
                            node_ll.append(vertex(vertex_index,node))
                        vertex_index += 1
                    adj_list.append(node_ll)
                    col_index += 1
                    
                adj_lists.append(adj_list)
                
        row_index+=1
            
    return adj_matrices,adj_lists

# Generate positive directed-graph with increasing elements and fixed vertices
# Returns graphs of fixed vertices growing in unconnected nodes
def generate_graphs_fixed_vertices(starting_graph_size, max_elements):
    adj_matrices = []
    adj_lists = []
    
    adj_matrix,adj_list = generate_graph(starting_graph_size)
    adj_matrices.append(adj_matrix.copy())
    
    for i in range(starting_graph_size, max_elements):
        adj_matrix = np.vstack((adj_matrix, np.zeros((1,i))))
        adj_matrix = np.hstack((adj_matrix, np.zeros((i+1,1))))
        adj_matrices.append(adj_matrix.copy())
        
        #Conver adj matrix to adj list
        adj_list = []
        col_index = 0
        for row in adj_matrix:
            node_ll = []
            
            vertex_index = 0
            for node in row:
                if node != 0:
                    node_ll.append(vertex(vertex_index,node))
                vertex_index +=1
                
            adj_list.append(node_ll)
            col_index += 1
        adj_lists.append(adj_list)

    return adj_matrices,adj_lists
        
#Generate positive directed-graph(S) 
#Generates {adj_matrices} and {adj_lists} of size {num_tables}
def generate_graphs(num_tables, starting_graph_size, increment):
    node_size = starting_graph_size
    adj_matrice = []
    adj_lists = []
    
    # starting node size must be > 5
    if starting_graph_size < 5:
        starting_graph_size = 5
    
    for i in range(num_tables):
        adj_matrix, adj_list = generate_graph(node_size)
        adj_matrice.append(adj_matrix)
        adj_lists.append(adj_list)
        node_size += increment 
        
    return adj_matrice, adj_lists

In [3]:
### Heap implementation
def extract_minimum(p_queue):
    minimum = p_queue[0]
    
    #Adjust node
    if(len(p_queue) > 1):
        # replace first node with last
        p_queue[0] = p_queue[-1]
        del p_queue[-1]
        
        # readjust priority queue 
        p_queue = shift_down(p_queue, 0)
    else:
        return minimum.index, []
    return minimum.index, p_queue
    
def insert(p_queue, vertex):
    #Insert Node to queue
    p_queue.append(vertex)
    
    #Shift to correct position
    if(len(p_queue) > 1):
        p_queue = shift_up(p_queue, len(p_queue)-1)
    
    return p_queue
    
def shift_up(p_queue, index):
    parent_index = (index-1) // 2
    
    #Iterate and node is at the correct position
    while(index > 0 and p_queue[parent_index].weight > p_queue[index].weight):
        #swap parent
        p_queue[parent_index], p_queue[index] = p_queue[index], p_queue[parent_index]
        
        #update current index
        index = parent_index
        parent_index = parent_index // 2
        
    return p_queue

def shift_down(p_queue, index):
    parent_index = index
    size = len(p_queue)-1
    
    #compare parent with left child
    left_index = (2 * index) + 1
    if(left_index <= size and p_queue[left_index].weight < p_queue[parent_index].weight):
        parent_index = left_index    
    
    #compare parent with right child
    right_index = (2 * index) + 2
    if(right_index <= size and p_queue[right_index].weight < p_queue[parent_index].weight):
        parent_index = right_index
    
    #shift parent down if it is less than child
    if(index != parent_index):
        p_queue[index],p_queue[parent_index] = p_queue[parent_index],p_queue[index]
        shift_down(p_queue, parent_index)
        
    return p_queue 

def remove(p_queue, vert_index):
    index = -1
    
    for i in range(len(p_queue)):
        if p_queue.index == vert_index:
            index = i
    
    if index != -1:
        p_queue[index] = p_queue[0]

        p_queue = shift_up(p_queue, index)

        return extract_minimum(p_queue)
    
    else:
        return p_queue

In [4]:
def dijkstra_shortestpath_alist(graph, start, end):
    d = [sys.maxsize for i in range(len(graph))]
    pi = [sys.maxsize for i in range(len(graph))]
    d[start] = 0
    # create priority queue
    p_queue = []
    
    # insert starting node as vertice
    for i in range(len(graph)):
         p_queue = insert(p_queue, vertex(i,d[i]))
    
    # find shortest path
    while(len(p_queue) != 0):
        u, p_queue = extract_minimum(p_queue)
        
        for vert in graph[u]:
            # check for shortest distance
            if(d[vert.index] > d[u] + vert.weight):
                d[vert.index] = d[u] + vert.weight
                pi[vert.index] = u
                #change priority
                p_queue = remove(p_queue, vert.index)
                p_queue = insert(p_queue, vertex(vert.index, d[vert.index]))
        
    return d

In [5]:
def dijkstra(graph, start):

    d= [float("inf") for _ in range(len(graph))]
    s = [0 for _ in range(len(graph))]
    d[start] = 0

    while True:
        shortest_distance = float("inf")
        shortest_index = -1
        for i in range(len(graph)):
            if d[i] < shortest_distance and  s[i]==0:
                shortest_distance = d[i]
                shortest_index = i
        if shortest_index == -1:
            return d
        for i in range(len(graph[shortest_index])):
            if graph[shortest_index][i] != 0 and d[i] > d[shortest_index] + graph[shortest_index][i]:
                d[i] = d[shortest_index] + graph[shortest_index][i]
        s[shortest_index] = True

# My part

In [6]:
def generate_compare_graph(num_elements):
    adj_matrices = []
    adj_lists = []

    # Generate Zero Matrix
    adj_matrix = np.zeros((num_elements,num_elements))
    row_index = 0
    for i in adj_matrix:
        for node in range(len(i)):
            if row_index != node:
                #Change current point vertice value to random number
                adj_matrix[row_index][node] = np.random.randint(1,num_elements)
                
        adj_matrices.append(adj_matrix.copy())

                #Convert adj matrix to adj list
        adj_list = []
        col_index = 0
        for row in adj_matrix:
            node_ll = []
            vertex_index = 0
            for node in row:
                if node != 0:
                    node_ll.append(vertex(vertex_index,node))
                vertex_index += 1
            adj_list.append(node_ll)
            col_index += 1
                    
        adj_lists.append(adj_list)
                
        row_index+=1
            
    return adj_matrices,adj_lists

In [7]:

timee = []
for i in range(100,501,50):
    vertice = i
    compare_matrix, compare_list = generate_compare_graph(vertice)
    no_edges = np.arange(vertice-1,vertice*(vertice-1)+1,vertice-1)
    
    for testcase in range(len(no_edges)):
        curr_list = compare_list[testcase]
        curr_matrix = compare_matrix[testcase]
        dest_node = len(curr_list)-1
    
    # Capture Compute Time
        start_time_list = time.time()
        dijkstra_shortestpath_alist(curr_list,0,dest_node)
        end_time_list = time.time()
    
        start_time_matrix = time.time()
        dijkstra(curr_matrix,0)
        end_time_matrix = time.time()
    
    # Capture Stats
        time_taken_list = (end_time_list - start_time_list)*1000
        time_taken_matrix = (end_time_matrix - start_time_matrix)*1000
    
        if time_taken_list > time_taken_matrix:
            timee.append(no_edges[testcase])
            break;
print(timee)


[693, 1341, 2985, 4233, 3887, 5584, 9177, 7184, 7984]


In [8]:
"""
vertice = 300
compare_matrix, compare_list = generate_compare_graph(vertice)
no_edges = np.arange(vertice-1,vertice*(vertice-1)+1,vertice-1)


time_taken_matrix = []
time_taken_list = []
for testcase in range(len(compare_list)):
    curr = compare_list[testcase]
    dest_node = len(curr)-1
    
    # Capture Compute Time
    start_time = time.time()
    dijkstra_shortestpath_alist(curr,0,dest_node)
    end_time = time.time()
    
    # Capture Stats
    time_taken = (end_time - start_time)*1000
    time_taken_list.append(time_taken)

for testcase in range(len(compare_matrix)):
    curr = compare_matrix[testcase]
    
    # Capture Compute Time
    start_time = time.time()
    dijkstra(curr,0)
    end_time = time.time()
    
    # Capture Stats
    time_taken = (end_time - start_time)*1000
    time_taken_matrix.append(time_taken)
"""

'\nvertice = 300\ncompare_matrix, compare_list = generate_compare_graph(vertice)\nno_edges = np.arange(vertice-1,vertice*(vertice-1)+1,vertice-1)\n\n\ntime_taken_matrix = []\ntime_taken_list = []\nfor testcase in range(len(compare_list)):\n    curr = compare_list[testcase]\n    dest_node = len(curr)-1\n    \n    # Capture Compute Time\n    start_time = time.time()\n    dijkstra_shortestpath_alist(curr,0,dest_node)\n    end_time = time.time()\n    \n    # Capture Stats\n    time_taken = (end_time - start_time)*1000\n    time_taken_list.append(time_taken)\n\nfor testcase in range(len(compare_matrix)):\n    curr = compare_matrix[testcase]\n    \n    # Capture Compute Time\n    start_time = time.time()\n    dijkstra(curr,0)\n    end_time = time.time()\n    \n    # Capture Stats\n    time_taken = (end_time - start_time)*1000\n    time_taken_matrix.append(time_taken)\n'

In [9]:
"""
fig = plt.figure()
ax = fig.add_subplot(1, 1, 1)
ax.plot(no_edges, time_taken_list, label = 'List')
ax.plot(no_edges, time_taken_matrix,label = 'Matrix')
ax.grid(True)
plt.xlabel('Number of Edges')
plt.ylabel('Times (ms)')
plt.legend()
plt.title('Elements vs Time (Fixed Vertices)')
plt.show()
"""

"\nfig = plt.figure()\nax = fig.add_subplot(1, 1, 1)\nax.plot(no_edges, time_taken_list, label = 'List')\nax.plot(no_edges, time_taken_matrix,label = 'Matrix')\nax.grid(True)\nplt.xlabel('Number of Edges')\nplt.ylabel('Times (ms)')\nplt.legend()\nplt.title('Elements vs Time (Fixed Vertices)')\nplt.show()\n"